#### Purpose:
Sets up new training, testing, and validation splits based on class directories of melspecs
- can contact author for S2L-ABGQI dataset otherwise requires separate mel spectrograms 

In [1]:
import glob
import os
import numpy as np
from sklearn.model_selection import train_test_split
import math
import random
import shutil

In [2]:
os.getcwd()

''

In [3]:
melspec_dir = '../../melspec_generation/results/melspecs/'
split_dirs = ['training','validation','testing']
new_split_dir = '../data/splits/'

# split definition n =
test_n = 50
val_n = 200 # interference is n = 50

In [4]:
# receives a source directoy (where png lives) and target (where to copy to) 
#    and uses str splicing [MAY NEED TO BE ADAPTED TO DIFFFERENT DATASETS or PCS] to get simple filenames
#    String splicing has to happen because of complicated naming convention from prior steps (Oct 2020)
def file_copier(src_path, dest_path, index):
    # string splicing
    ext = src_path.split("_")[-1] # get extension
    cl = src_path.split("_")[-5] # get class
    cl_annotation = cl.split("\\")[-1] # clean to class annotation
    
    # target directory
    temp_dest = dest_path + "/" + str(index) + "_" + cl_annotation + "_" + ext
    
    # copy png
    shutil.copy(src_path, temp_dest)

In [5]:
# retrieve class
class_dirs = glob.glob(melspec_dir + "*")
class_dirs = [i.split("\\")[1] for i in class_dirs]
class_dirs

['AMA', 'AMU', 'AVT', 'BBI', 'OQU']

In [6]:
# building tr/ts/val directory structure with classes in each split 

# build parent dir
if(os.path.exists(os.path.join(new_split_dir))):
    print("new split dir exists....")
else:
    os.mkdir(os.path.join(new_split_dir))
    print("built new dir :" + new_split_dir)

new split dir exists....


In [7]:
for split in split_dirs:
    # build training, testing, validation dirs
    if(os.path.exists(os.path.join(new_split_dir, split))):
        print(split, "directory already exists. Skipping...")
    else:
        os.mkdir(os.path.join(new_split_dir, split))

    for i in range(len(class_dirs)):
        # build class dirs inside split dirs
        if(os.path.exists(os.path.join(new_split_dir, split, class_dirs[i]))):
            print(split_dirs[i], "directory already exists. Skipping...")
        else:
            os.mkdir(os.path.join(new_split_dir, split, class_dirs[i]))

print("Created melspec dirs:")
print(os.listdir(new_split_dir))

training directory already exists. Skipping...
validation directory already exists. Skipping...
testing directory already exists. Skipping...
Created melspec dirs:
['testing', 'training', 'validation']


In [8]:
split_dirs = glob.glob(new_split_dir + "*")
split_dirs

['../data/splits\\testing',
 '../data/splits\\training',
 '../data/splits\\validation']

In [17]:
len(files)

1

In [9]:
# iterate thru each class directory of mel specs
for i, temp in enumerate(class_dirs):
    print()
    class_dir = temp # class name
    temp_dir = melspec_dir + temp # full class path name 
    print(temp_dir)
    files = glob.glob(temp_dir + "/*.png")  # list mel pngs in current class dir
    tot_sam = len(files) # how many spectrograms do we have
    
    # data split for ABGQ
    if temp != 'interference':
        tr_sam = math.ceil(tot_sam - test_n - val_n) # account for test and val data
        val_sam = val_n
        test_sam = test_n
        
    # adjust for interference because 200 validation samples would result in insufficient training samples
    else:
        val_sam = 50
        test_sam = 50
        tr_sam = math.ceil(tot_sam - test_sam - val_sam)
       
    # print out what our splits will look like 
    print("Tot-Sam:", tot_sam)
    print('Tot training samples =', tr_sam) 
    print('Tot validation samples =', val_sam) 
    print('Tot testing samples = ', test_sam)
    
    # create list the length of the target class ROIs
    values = list(range(tot_sam))    
    # randomize values
    random.shuffle(values)
    
    # grab upto specified training samples then testing samples then val smaples from ranomized list
    train = values[:tr_sam]
    test = values[tr_sam:(tr_sam + test_sam)]
    val = values[-val_sam:]
    
    # grab correpsonding file indices and *COPY* image to correct split directory
    # copy training
    for i in train:
        dest_path = new_split_dir + "training/" + class_dir
        file_copier(files[i], dest_path, i)
        
    # copy testing
    for j in test:
        dest_path = new_split_dir + "testing/" + class_dir
        file_copier(files[j], dest_path, j)
    
    # copy validation
    for k in val:
        dest_path = new_split_dir + "validation/" + class_dir
        file_copier(files[k], dest_path, k)


../../melspec_generation/results/melspecs/AMA
Tot-Sam: 1
Tot training samples = -249
Tot validation samples = 200
Tot testing samples =  50


IndexError: list index out of range